<a href="https://colab.research.google.com/github/8ightdon/ai-week-1-assignment-/blob/main/Crypto_ChainChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# chatbot_app.py

# Install necessary packages
!pip install flask flask-ngrok requests

# --------------------------
# chatbot.py (Rule-based logic)
# --------------------------
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
import requests

class RuleBasedChatbot:
    def __init__(self):
        self.rules = {
            "hi": "Hello! How can I assist you today?",
            "hello": "Hi there! What can I help you with?",
            "help": "Try asking about Bitcoin, Ethereum, or say hi!",
            "bye": "Goodbye! See you again soon.",
        }
        self.supported_coins = ["bitcoin", "ethereum", "solana", "cardano", "dogecoin"]

    def get_crypto_data(self, coin):
        url = f"https://api.coingecko.com/api/v3/coins/{coin.lower()}"
        response = requests.get(url)
        if response.status_code != 200:
            return f"❌ Could not fetch data for {coin}."

        data = response.json()
        market_data = data.get("market_data", {})
        price_change = market_data.get("price_change_percentage_30d", "N/A")
        energy_efficiency = data.get("hashing_algorithm", "Unknown")
        sustainability_note = "Moderate" if "proof" in str(energy_efficiency).lower() else "Efficient"

        return (
            f"📊 {coin.capitalize()} Analysis:\n"
            f"- 30d Price Change: {price_change}%\n"
            f"- Energy Model: {energy_efficiency}\n"
            f"- Sustainability: {sustainability_note}"
        )

    def respond(self, message):
        message = message.lower().strip()
        if message in self.rules:
            return self.rules[message]
        elif message in self.supported_coins:
            return self.get_crypto_data(message)
        return "🤖 I don't understand that. Try 'help'."

# --------------------------
# Flask Web App
# --------------------------
app = Flask(__name__)
run_with_ngrok(app)

chatbot = RuleBasedChatbot()

html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>CryptoChatbot</title>
    <style>
        body { font-family: Arial; padding: 20px; max-width: 600px; margin: auto; background: #f9f9f9; }
        h1 { color: #333; }
        #chat { border: 1px solid #ccc; padding: 10px; height: 300px; overflow-y: scroll; background: #fff; }
        input, button { padding: 10px; width: 80%; margin-top: 10px; }
        button { width: 18%; }
    </style>
</head>
<body>
    <h1>💬 CryptoChain Chatbot</h1>
    <div id="chat"></div>
    <input id="message" placeholder="Type your message..." />
    <button onclick="send()">Send</button>
    <script>
        async function send() {
            let msg = document.getElementById("message").value;
            document.getElementById("chat").innerHTML += "<p><b>You:</b> " + msg + "</p>";
            const res = await fetch("/chat", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ message: msg })
            });
            const data = await res.json();
            document.getElementById("chat").innerHTML += "<p><b>Bot:</b> " + data.response.replace(/\\n/g, "<br>") + "</p>";
            document.getElementById("message").value = "";
            document.getElementById("chat").scrollTop = document.getElementById("chat").scrollHeight;
        }
    </script>
</body>
</html>
"""

@app.route("/")
def home():
    return render_template_string(html_template)

@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.json.get("message", "")
    response = chatbot.respond(user_input)
    return jsonify({"response": response})

# --------------------------
# Run the app
# --------------------------
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
import requests

# 🔄 Fetch real-time crypto data from CoinGecko
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'ids': 'bitcoin,ethereum,cardano',
        'order': 'market_cap_desc',
        'per_page': 3,
        'page': 1,
        'sparkline': False
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        data = response.json()

        crypto_db = {}
        for coin in data:
            coin_id = coin["id"]
            name = coin["name"]
            sustainability = {"bitcoin": 3, "ethereum": 6, "cardano": 8}[coin_id]
            energy_use = {"bitcoin": "high", "ethereum": "medium", "cardano": "low"}[coin_id]

            crypto_db[name] = {
                "price": coin["current_price"],
                "price_trend": "rising" if coin["price_change_percentage_24h"] >= 0 else "falling",
                "market_cap": "high" if coin["market_cap_rank"] <= 3 else "medium",
                "energy_use": energy_use,
                "sustainability_score": sustainability
            }

        return crypto_db

    except Exception as e:
        print("❌ Error fetching data from CoinGecko:", e)
        return {}

# 💬 Main chatbot function
def crypto_chatbot():
    print("👋 Hi, I’m CryptoChain Chat — your AI-powered financial sidekick!")
    print("Ask me about trending coins, sustainability, or profits. Type 'exit' to quit.\n")

    crypto_db = fetch_crypto_data()

    if not crypto_db:
        print("⚠️ Unable to start the chatbot due to data loading error.")
        return

    while True:
        user_query = input("You: ").lower()

        if user_query in ['exit', 'quit']:
            print("CryptoChain Chat: Stay sharp! Crypto is risky—always do your own research. 🚀")
            break

        elif "trending" in user_query or "up" in user_query:
            trending = [name for name, data in crypto_db.items() if data["price_trend"] == "rising"]
            if trending:
                print(f"📈 Trending up: {', '.join(trending)}")
            else:
                print("CryptoChain Chat: No coins are trending up right now.")

        elif "sustainable" in user_query:
            best = max(crypto_db, key=lambda x: crypto_db[x]["sustainability_score"])
            score = crypto_db[best]['sustainability_score']
            print(f"🌱 {best} is the most sustainable choice with a score of {score}/10.")

        elif "profitable" in user_query or "growth" in user_query or "investment" in user_query:
            profitable = [name for name, data in crypto_db.items()
                          if data["price_trend"] == "rising" and data["market_cap"] == "high"]
            if profitable:
                print(f"💰 Most profitable now: {', '.join(profitable)}")
            else:
                print("CryptoChain Chat: No highly profitable coins at the moment.")

        elif "energy" in user_query or "eco" in user_query:
            eco_friendly = [name for name, data in crypto_db.items() if data["energy_use"] == "low"]
            if eco_friendly:
                print(f"⚡️ Low energy coins: {', '.join(eco_friendly)}")
            else:
                print("CryptoChain Chat: No coins with low energy usage right now.")

        else:
            print("🤖 I didn’t get that. Try asking about 'trending', 'sustainable', or 'profitable' coins.")

# ▶️ Run the bot
crypto_chatbot()


👋 Hi, I’m CryptoChain Chat — your AI-powered financial sidekick!
Ask me about trending coins, sustainability, or profits. Type 'exit' to quit.

You: bitcoin
🤖 I didn’t get that. Try asking about 'trending', 'sustainable', or 'profitable' coins.
You: trending
CryptoChain Chat: No coins are trending up right now.
You: sustainable
🌱 Cardano is the most sustainable choice with a score of 8/10.
You: profitable
CryptoChain Chat: No highly profitable coins at the moment.
